In [7]:
from citipy import citipy
import pandas as pd
import numpy as np
import json 
import requests
import matplotlib.pyplot as plt
from config import gkey
from config import api_key
from pprint import pprint
from scipy import stats as st
import gmaps

In [8]:
#Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [9]:
#building my cities_df from my data of WeatherPy
file = "resources/cleancities.csv"
cities_df = pd.read_csv(file)
cities_df.head()

,lat,long,City,Country,Temp,Humidity,Cloudiness,Wind_Speed
0,6.1241,81.1185,hambantota,lk,78.80,89.0,18.0,11.50
1,-33.9180,25.5701,port elizabeth,za,64.40,82.0,92.5,10.36
2,-33.6500,115.3333,busselton,au,80.01,45.0,37.0,4.43
3,-33.6971,-53.4616,chuy,uy,71.29,96.0,5.0,6.04
4,37.5683,126.9778,seoul,kr,50.00,53.0,5.0,8.05


In [27]:
#The highest humidity as required
cities_df["Humidity"].max()

100.0

In [28]:
#The location of the highest humidity
hi_humi_df = cities_df.loc[cities_df["Humidity"] == 100, :]
hi_humi_df

,lat,long,City,Country,Temp,Humidity,Cloudiness,Wind_Speed
57,-3.7703,-70.3831,puerto narino,co,77.00,100.0,18.0,0.60
96,38.7992,-9.4469,colares,pt,58.37,100.0,92.5,19.19
242,46.1667,1.8667,gueret,fr,33.94,100.0,5.0,4.61
246,-24.5094,-48.8425,apiai,br,63.14,100.0,92.5,1.57
285,14.5094,-92.1933,ocos,gt,69.58,100.0,5.0,7.58
475,53.8000,-9.5167,westport,ie,30.00,100.0,5.0,4.00
483,38.7333,-27.0667,praia da vitoria,pt,63.36,100.0,67.5,25.39
491,61.5814,-149.4394,wasilla,us,31.21,100.0,92.5,2.21
495,-38.7000,-73.1667,carahue,cl,48.20,100.0,5.0,3.44


In [10]:
#Prepare locations and humidity data for heatmap
locations = cities_df[["lat", "long"]]
humidity = cities_df["Humidity"]

In [13]:
#The heatmap for world cities vs humidity
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights= humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
#select of vacation places about 9 cities:
vac_df = cities_df.loc[(cities_df["Temp"] > 70) & (cities_df["Temp"] < 80) & (cities_df["Wind_Speed"] < 5) & 
                       (cities_df["Cloudiness"] <= 5.0), :]

In [21]:
vac_df.head(10)

,lat,long,City,Country,Temp,Humidity,Cloudiness,Wind_Speed
43,-12.9319,45.1483,chirongui,yt,77.00,88.0,5.0,2.30
139,6.3005,-10.7969,monrovia,lr,76.59,87.0,5.0,3.80
152,-12.5763,13.4055,benguela,ao,72.55,78.0,5.0,4.05
179,9.5380,-13.6773,conakry,gn,75.42,75.0,5.0,3.85
198,18.0000,79.5833,warangal,in,70.81,62.0,5.0,1.70
295,4.8699,-2.2405,axim,gh,76.87,86.0,5.0,4.38
332,17.9583,-102.2000,lazaro cardenas,mx,73.67,83.0,5.0,3.38
397,7.5264,-12.5050,bonthe,sl,74.97,82.0,5.0,3.24
520,14.1059,-15.5508,kaffrine,sn,71.60,15.0,5.0,4.61


In [23]:
vac_locations = vac_df[["lat", "long"]]

In [25]:
# Create city marker layer and mark my vacation locations (totally 9)
markers = gmaps.marker_layer(vac_locations, )
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))

In [90]:
hotel_df = vac_df.loc[:,["lat", "long", "City", "Country"]]
hotel_df["Hotel"] = ""
hotel_df["Address"] =""
hotel_df["Rating"] = ""
#hotel_df.to_csv("resources/hotel.csv")
hotel_df.reset_index(inplace = True)
hotel_df.head(10)

,index,lat,long,City,Country,Hotel,Address,Rating
0,43,-12.9319,45.1483,chirongui,yt,,,
1,139,6.3005,-10.7969,monrovia,lr,,,
2,152,-12.5763,13.4055,benguela,ao,,,
3,179,9.5380,-13.6773,conakry,gn,,,
4,198,18.0000,79.5833,warangal,in,,,
5,295,4.8699,-2.2405,axim,gh,,,
6,332,17.9583,-102.2000,lazaro cardenas,mx,,,
7,397,7.5264,-12.5050,bonthe,sl,,,
8,520,14.1059,-15.5508,kaffrine,sn,,,


In [91]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [92]:
#build params
target_search = "hotel"
target_radius = 5000
target_type = "hotel"
lat = hotel_df["lat"].tolist()
long = hotel_df["long"].tolist() 


In [93]:
hotel = []
add = []
rating = []

In [94]:
#To get the hotel info

for index, row in hotel_df.iterrows():
    try:
    
        target_coordinates = f"{lat[index]}, {long[index]}"
        #print(target_coordinates)
        params = {"location": target_coordinates,
                    "keyword": target_search,
                    "radius": target_radius,
                    "type": target_type,
                    "key": gkey}

        # run a request using our params dictionary
        response = requests.get(base_url, params = params)
        r = response.json()
        #pprint(r)
        name = r["results"][0]["name"]
        #print(name)
        addr = r["results"][0]["vicinity"]
        #print(addr)
        rat = r["results"][0]["rating"]
        #print(rat)
        hotel.append(name)
        add.append(addr)
        rating.append(rat)
        #print(f"{index},{row["City"]}'s hotel is done")
    except:
        #print(f"{index}, {row["City"]}'s hotel is not find")
        pass

-12.9319, 45.1483
Les Baobabs
Bandrele
4.4
6.3005, -10.7969
The Royal Grand Hotel
Tubman Blvd, Monrovia
4.3
-12.5763, 13.4055
Aparthotel Mil Cidades
R. Aires de Almeida Santos, Benguela
4.2
9.538, -13.6773
Noom Hotel Conakry
Teminetaye, Ave de la Republique, Kaloum
4.1
18.0, 79.5833
Hotel Grand Gayathri
8-5, 49/A, Station Rd, Opposite Sunil Theatre, Girmajipet, Warangal
4
4.8699, -2.2405
Ankobra Beach
Old Axim Road 1, Axim
4
17.9583, -102.2
City Express Lázaro Cárdenas
Av. de las Palmas No. 1291 Lote 2 Mza. 1 Octavo Sector, Centro, Lázaro Cárdenas
4.1
7.5264, -12.505
Bonthe Holiday Village
Bonthe
4.5
14.1059, -15.5508
Hotel de l'Auberge
N1, Kaffrine
2.9


In [97]:
#The final hotels DataFrame with all the information listed
hotel_df["Hotel"] = hotel
hotel_df["Address"] = add
hotel_df["Rating"] = rating
hotel_df.to_csv("results/hotel.csv")
hotel_df.head(10)

,index,lat,long,City,Country,Hotel,Address,Rating
0,43,-12.9319,45.1483,chirongui,yt,Les Baobabs,Bandrele,4.4
1,139,6.3005,-10.7969,monrovia,lr,The Royal Grand Hotel,"Tubman Blvd, Monrovia",4.3
2,152,-12.5763,13.4055,benguela,ao,Aparthotel Mil Cidades,"R. Aires de Almeida Santos, Benguela",4.2
3,179,9.5380,-13.6773,conakry,gn,Noom Hotel Conakry,"Teminetaye, Ave de la Republique, Kaloum",4.1
4,198,18.0000,79.5833,warangal,in,Hotel Grand Gayathri,"8-5, 49/A, Station Rd, Opposite Sunil Theatre,...",4.0
5,295,4.8699,-2.2405,axim,gh,Ankobra Beach,"Old Axim Road 1, Axim",4.0
6,332,17.9583,-102.2000,lazaro cardenas,mx,City Express Lázaro Cárdenas,Av. de las Palmas No. 1291 Lote 2 Mza. 1 Octav...,4.1
7,397,7.5264,-12.5050,bonthe,sl,Bonthe Holiday Village,Bonthe,4.5
8,520,14.1059,-15.5508,kaffrine,sn,Hotel de l'Auberge,"N1, Kaffrine",2.9
